# Задание 2
Используйте kernel trick для классификации графов с помощью ridge регрессии (она используется для регрессии, но здесь мы сделаем вид, что все ок, и будем обрубать значения меньше 0 и больше 1) и svm (в Sklearn можно использовать кастомные kernels).
Датасеты отсюда: https://github.com/FilippoMB/Benchmark_dataset_for_graph_classification
Ядра можно взять отсюда: https://github.com/ysig/GraKeL
Подберите лучшее ядро для всех случаев.

In [56]:
import numpy as np
import networkx as nx
from grakel import GraphKernel, graph_from_networkx
from sklearn import svm
from sklearn.metrics import accuracy_score
from time import time

In [57]:
# Хотим изменить все значения так, чтобы они были из промежутка [0, 1]
def clip_values(arr):
    arr = np.array(arr)
    return np.clip(arr, 0, 1)

# Проверяем, что данные ок
def is_valid_data(arr):
    arr = np.array(arr)
    return not (np.isnan(arr).any() or np.isinf(arr).any())

In [58]:
# загружаем данные из датасета hard_small, тк на датасете hard все алгоритмы очень долго работают
# поскольку в нем очень много значений

# загружаю данные по официальному примеру с репы grakel
loaded = np.load('hard_small.npz', allow_pickle=True)
A_train = list(loaded['tr_adj'])
X_train = loaded['tr_feat']
y_train = loaded['tr_class']
A_test = list(loaded['te_adj'])
X_test = loaded['te_feat']
y_test = loaded['te_class']

# Проверяем данные на валидность
X_train = [clip_values(x) for x in X_train if is_valid_data(x)]
X_test = [clip_values(x) for x in X_test if is_valid_data(x)]

# Конвертация в формат networkx также по примеру кода из репы grakel
G_tr = []
for a, x in zip(A_train, X_train):
    G = nx.from_scipy_sparse_array(a)
    x_tuple = tuple(map(tuple, x))
    nx.set_node_attributes(G, dict(enumerate(x_tuple)), 'features')
    G_tr.append(G)

G_te = []
for a, x in zip(A_test, X_test):
    G = nx.from_scipy_sparse_array(a)
    x_tuple = tuple(map(tuple, x))
    nx.set_node_attributes(G, dict(enumerate(x_tuple)), 'features')
    G_te.append(G)

In [59]:
G_train = graph_from_networkx(G_tr, node_labels_tag='features')
G_train = [g for g in G_train]
y_train = np.argmax(y_train, axis=-1)
G_test = graph_from_networkx(G_te, node_labels_tag='features')
G_test = [g for g in G_test]
y_test = np.argmax(y_test, axis=-1)

In [60]:
kernel_names = [
    "shortest_path",
    "graphlet_sampling",
    "pyramid_match",
    "svm_theta",
    "neighborhood_hash",
    "subtree_wl",
    "odd_sth",
    "propagation",
    "vertex_histogram",
    "weisfeiler_lehman",
    "core_framework"
]

In [61]:
for k_ in kernel_names:  # просто перебираем все ядра
    try:
        start = time()
        if k_ in ["weisfeiler_lehman", "core_framework"]:
            gk = GraphKernel(kernel=[{"name": k_}, {"name": "shortest_path"}], normalize=True)

        else:
            gk = GraphKernel(kernel=[{"name": k_}], normalize=True)

        K_train = gk.fit_transform(G_train)
        K_test = gk.transform(G_test)
        if not is_valid_data(K_train) or not is_valid_data(K_test):
            print(f"Пропущено ядро {k_} из-за некорректных значений в матрице ядра.")
            continue
        clf = svm.SVC(kernel='precomputed', C=1)
        clf.fit(K_train, y_train)
        y_pred = clf.predict(K_test)
        acc = accuracy_score(y_test, y_pred)
        end = time()
        print(k_, "-- Accuracy:", acc, " time",
              str(round(end - start, 2)), "s")
    except Exception as e:
        print(f"Ошибка при обработке ядра {k_}: {e}")

shortest_path -- Accuracy: 0.6923076923076923  time 2.8 s
graphlet_sampling -- Accuracy: 0.38461538461538464  time 13.04 s
pyramid_match -- Accuracy: 0.23076923076923078  time 0.98 s


/Users/elizabeth/anaconda3/lib/python3.11/site-packages/grakel/kernels/kernel.py:202: RuntimeWarning: invalid value encountered in divide
  return km / np.sqrt(np.outer(self._X_diag, self._X_diag))
/Users/elizabeth/anaconda3/lib/python3.11/site-packages/grakel/kernels/kernel.py:167: RuntimeWarning: invalid value encountered in divide
  km /= np.sqrt(np.outer(Y_diag, X_diag))


Пропущено ядро svm_theta из-за некорректных значений в матрице ядра.
neighborhood_hash -- Accuracy: 0.6923076923076923  time 1.09 s
subtree_wl -- Accuracy: 0.15384615384615385  time 0.0 s
odd_sth -- Accuracy: 0.4230769230769231  time 8.21 s
propagation -- Accuracy: 0.5384615384615384  time 1.06 s
vertex_histogram -- Accuracy: 0.15384615384615385  time 0.01 s
weisfeiler_lehman -- Accuracy: 0.7307692307692307  time 23.0 s
core_framework -- Accuracy: 0.6923076923076923  time 5.7 s


Видно, что самые лучшие результаты выдает weisfeiler_lehman, но он работает дольше всего, в соотношении время - качество, лучше всего работает shortest_path

Проверим, какие будут результаты у Ridge - регрессии

In [62]:
from sklearn.linear_model import Ridge

# делаю то же самое, но теперь для ridge
for k_ in kernel_names:
    try:
        start = time()
        if k_ in ["weisfeiler_lehman", "core_framework"]:
            gk = GraphKernel(kernel=[{"name": k_}, {"name": "shortest_path"}], normalize=True)

        else:
            gk = GraphKernel(kernel=[{"name": k_}], normalize=True)

        K_train = gk.fit_transform(G_train)
        K_test = gk.transform(G_test)

        if not is_valid_data(K_train) or not is_valid_data(K_test):
            print(f"Пропущено ядро {k_} из-за некорректных значений в матрице ядра.")
            continue

        ridge = Ridge(alpha=1.0)
        ridge.fit(K_train, y_train)

        y_pred = ridge.predict(K_test)
        y_pred = clip_values(y_pred)
        y_pred_class = np.round(y_pred)
        acc = accuracy_score(y_test, y_pred_class)
        end = time()
        print(k_, "-- Accuracy:", acc, " time",
              str(round(end - start, 2)), "s")
    except Exception as e:
        print(f"Ошибка при обработке ядра {k_}: {e}")


shortest_path -- Accuracy: 0.4230769230769231  time 2.83 s
graphlet_sampling -- Accuracy: 0.34615384615384615  time 11.97 s
pyramid_match -- Accuracy: 0.34615384615384615  time 0.97 s


/Users/elizabeth/anaconda3/lib/python3.11/site-packages/grakel/kernels/kernel.py:202: RuntimeWarning: invalid value encountered in divide
  return km / np.sqrt(np.outer(self._X_diag, self._X_diag))
/Users/elizabeth/anaconda3/lib/python3.11/site-packages/grakel/kernels/kernel.py:167: RuntimeWarning: invalid value encountered in divide
  km /= np.sqrt(np.outer(Y_diag, X_diag))


Пропущено ядро svm_theta из-за некорректных значений в матрице ядра.
neighborhood_hash -- Accuracy: 0.34615384615384615  time 0.88 s
subtree_wl -- Accuracy: 0.34615384615384615  time 0.01 s
odd_sth -- Accuracy: 0.34615384615384615  time 7.16 s
propagation -- Accuracy: 0.34615384615384615  time 0.95 s
vertex_histogram -- Accuracy: 0.34615384615384615  time 0.01 s
weisfeiler_lehman -- Accuracy: 0.4230769230769231  time 21.39 s
core_framework -- Accuracy: 0.4230769230769231  time 5.77 s


shortest_path, core_framework, weisfeiler_lehman выдают одинаковые результаты, но быстрее всего отрабатывает shortest_path

Дополнительно: поэкспериментируйте с kernel construction на основе kernels из библиотеки и попробуйте предложить свой kernel, который работает лучше

In [63]:
from sklearn.model_selection import train_test_split
G_train, G_test, y_train, y_test = train_test_split(G_train, y_train, test_size=0.3, random_state=42)
# перемешиваю данные
# без этого у меня получались подозрительно одинаковые ответы всегда, получалось accuracy = accuracy weisfeiler_lehman для svm

In [75]:
class CustomKernel:  # класс для создания собственного ядра, можно брать взвешенную сумму ядер (линейная функция)
    # а можно брать нелинейную функцию
    def __init__(self, normalize=True, combination='sum', weights=None):
        self.gk1 = None
        self.gk2 = None
        self.gk3 = None
        self.gk4 = None
        self.K_train = None
        self.normalize = normalize
        self.combination = combination  # лк, произведение
        self.weights = weights if weights is not None else [0.5, 0.5]

    def fit_transform(self, graphs):
        # Пробовала комбинировать kernels, которые давали лучшие результаты, но здесь сильно улучшить accuracy
        # не получалось. В связи с этим решила попробовать взять те, у которых были результаты не очень
        # У pyramid_match acc=0.23, у graphlet_sampling acc=0.35
        self.gk1 = GraphKernel(kernel=[{"name": "pyramid_match"}], normalize=self.normalize)
        self.gk2 = GraphKernel(kernel=[{"name": "graphlet_sampling"}], normalize=self.normalize)

        k1 = self.gk1.fit_transform(graphs)
        k2 = self.gk2.fit_transform(graphs)

        if self.combination == 'sum':
            # взвешенная сумма двух ядер (линейная комбинация)
            self.K_train = self.weights[0] * k1 + self.weights[1] * k2
        elif self.combination == 'multiply':
            # посмотрим еще на произведение
            self.K_train = k1 * k2
        return self.K_train

    def transform(self, graphs):
        k1 = self.gk1.transform(graphs)
        k2 = self.gk2.transform(graphs)

        if self.combination == 'sum':
            k_test = self.weights[0] * k1 + self.weights[1] * k2
        elif self.combination == 'multiply':
            k_test = k1 * k2
        return k_test

In [76]:
# Попробуем подобрать лучшие значения весов + посмотреть, как меняется accuracy
# при изменении весов
try:
    for w1 in np.arange(0.1, 1.0, 0.1):
        w2 = 1.0 - round(w1, 1)
        start = time()  # чтобы засечь время работы
        custom_kernel = CustomKernel(normalize=True, combination='sum', weights=[w1, w2])
        K_train_custom = custom_kernel.fit_transform(G_train)
        K_test_custom = custom_kernel.transform(G_test)

        if not is_valid_data(K_train_custom) or not is_valid_data(K_test_custom):
            print(f"Пропущено ядро (веса {w1}, {w2}) из-за некорректных значений в матрице.")
            continue

        clf = svm.SVC(kernel='precomputed', C=1, class_weight='balanced')  # вот здесь оказалось, что обязательно
        # надо добавлять class_weight='balanced' для правильного распределения данных
        clf.fit(K_train_custom, y_train)
        y_pred_custom = clf.predict(K_test_custom)
        acc_custom = accuracy_score(y_test, y_pred_custom)
        end = time()
        print(f"CustomKernel (weights {round(w1, 2)}, {round(w2, 2)}) -- Accuracy: {acc_custom}  time: {round(end - start, 2)} s") # округляю все, чтобы был красивый вывод,
        # тк в питоне float операции дают неточные ответы (знаем, что 0,1+0,2=0,30000000000000004)
except Exception as e:
    print(f"Ошибка: {e}")

CustomKernel (weights 0.1, 0.9) -- Accuracy: 0.43243243243243246  time: 11.0 s
CustomKernel (weights 0.2, 0.8) -- Accuracy: 0.43243243243243246  time: 10.44 s
CustomKernel (weights 0.3, 0.7) -- Accuracy: 0.4594594594594595  time: 10.37 s
CustomKernel (weights 0.4, 0.6) -- Accuracy: 0.5  time: 10.37 s
CustomKernel (weights 0.5, 0.5) -- Accuracy: 0.5135135135135135  time: 10.37 s
CustomKernel (weights 0.6, 0.4) -- Accuracy: 0.5135135135135135  time: 10.36 s
CustomKernel (weights 0.7, 0.3) -- Accuracy: 0.5  time: 10.35 s
CustomKernel (weights 0.8, 0.2) -- Accuracy: 0.43243243243243246  time: 10.32 s
CustomKernel (weights 0.9, 0.1) -- Accuracy: 0.40540540540540543  time: 10.4 s


Среди линейных комбинаций с различными весами лучшие результаты получаются, когда берем веса (сначала вес pyramid_match, потом вес graphlet_sampling) 0.5, 0.5; 0.6, 0.4 (результаты получаются одинаковыми, тк svm не чувствителен к небольшим изменениям в матрице ядра, то есть если матрицы имеют схожие значения, то эти веса будут давать похожие ответы.
С увеличением веса pyramid_match accuracy_score растет до веса=0.6, потом начинает падать
Это может быть связано с тем, что при резкой разнице между весами ядер у нас одно доминирует над другим, то есть (по крайней мере при крайних значениях) мы почти полностью игнорируем одно из ядер.


In [77]:
# считаем accuracy для произведения ядер
try:
    start = time()  # чтобы засечь время работы
    custom_kernel = CustomKernel(normalize=True, combination='multiply')
    K_train_custom = custom_kernel.fit_transform(G_train)
    K_test_custom = custom_kernel.transform(G_test)

    if not is_valid_data(K_train_custom) or not is_valid_data(K_test_custom):
        print("Пропущено из-за некорректных значений в матрице.")
    else:
        clf = svm.SVC(kernel='precomputed', C=1, class_weight='balanced')
        clf.fit(K_train_custom, y_train)
        y_pred_custom = clf.predict(K_test_custom)
        acc_custom = accuracy_score(y_test, y_pred_custom)
        end = time()
        print("CustomKernel (multiply) -- Accuracy:", acc_custom, " time", str(round(end - start, 2)), "s")
except Exception as e:
    print(f"Ошибка: {e}")

CustomKernel (multiply) -- Accuracy: 0.5405405405405406  time 10.4 s


Для pyramid_match и graphlet_sampling лучше всего получается результат, когда берем не линейную комбинацию ядер, а произведение. Это связано с тем, что произведение ядер усиливает сходства, то есть мы особенно сильно смотрим на области, где оба ядра pyramid_match и graphlet_sampling дают сильный сигнал. Если у нас есть какие-то сложные связи в графах, то поэтому может быть лучший результат у произведения.

По времени они все работают примерно одинаково, но дольше чем каждое ядро, взятое по отдельности.
Это связано с тем, что надо вычислять несколько ядер, а также с тем, что мы тратим время на операции на комбинирование + нормализацию

In [84]:
class LogKernel:
    def __init__(self, normalize=True, weights=None):
        self.normalize = normalize  # нормализация
        self.weights = weights if weights is not None else [0.5, 0.5]  # по дефолту веса по 1/2

    def fit_transform_pair(self, graphs, gk1, gk2):
        # преобразуем обучающую выборку
        k1 = gk1.fit_transform(graphs)
        k2 = gk2.fit_transform(graphs)
        return self.weights[0] * np.log(k1) + self.weights[1] * np.log(k2)  # логарифмическое ядро

    def transform_pair(self, graphs, gk1, gk2):
        # преобразуем тестовую выборку
        k1 = gk1.transform(graphs)
        k2 = gk2.transform(graphs)
        return self.weights[0] * np.log(k1) + self.weights[1] * np.log(k2)

In [85]:
try:
    start = time()  # замеряем время
    kernel_names = ["pyramid_match", "graphlet_sampling", "weisfeiler_lehman", "shortest_path"]
    # проинициализируем ядра
    kernels = [GraphKernel(kernel=[{"name": name}], normalize=True) for name in kernel_names]

    log_kernel = LogKernel(normalize=True)

    for i in range(len(kernels)):
        for j in range(i + 1, len(kernels)):  # хотим перебрать все пары из 4х
            gk1, gk2 = kernels[i], kernels[j]
            kernel1, kernel2 = kernel_names[i], kernel_names[j]

            K_train_custom = log_kernel.fit_transform_pair(G_train, gk1, gk2)
            K_test_custom = log_kernel.transform_pair(G_test, gk1, gk2)

            if not is_valid_data(K_train_custom) or not is_valid_data(K_test_custom):
                print(f"Пропущено ядро для ({kernel1}, {kernel2}) из-за некорректных значений.")
                continue

            clf = svm.SVC(kernel='precomputed', C=1, class_weight='balanced')
            clf.fit(K_train_custom, y_train)  # обучаем
            y_pred_custom = clf.predict(K_test_custom)  # предиктим
            acc_custom = accuracy_score(y_test, y_pred_custom)  # считаем скор
            end = time()

            print(f"Log Kernel Pair ({kernel1}, {kernel2}) -- Accuracy: {acc_custom}  time: {round(end - start, 2)} s")
except Exception as e:
    print(f"Ошибка: {e}")

Log Kernel Pair (pyramid_match, graphlet_sampling) -- Accuracy: 0.5405405405405406  time: 10.55 s
Log Kernel Pair (pyramid_match, weisfeiler_lehman) -- Accuracy: 0.47297297297297297  time: 11.59 s
Log Kernel Pair (pyramid_match, shortest_path) -- Accuracy: 0.5675675675675675  time: 14.82 s
Log Kernel Pair (graphlet_sampling, weisfeiler_lehman) -- Accuracy: 0.44594594594594594  time: 24.86 s
Log Kernel Pair (graphlet_sampling, shortest_path) -- Accuracy: 0.5945945945945946  time: 37.35 s
Log Kernel Pair (weisfeiler_lehman, shortest_path) -- Accuracy: 0.5945945945945946  time: 40.24 s


Заметим, что результаты не сильно улучшились по сравнению с ядрами без модификаций. Логарифмические ядра, где участвует weisfeiler_lehman выдают не очень хорошие результаты, тк мы делаем сильную редукцию значений, а для weisfeiler_lehman, например, важно, чтобы были различия в значениях. Поэтому, выгоднее брать это ядро в комбинациях, которые не делают сглаживания значений. Также я брала в комбинации ядра с не очень хорошим скором изначально, например у pyramid_match был результат всего 0.23, хотя в комбинации, например с shortest_path результат стал получше (shortest_path улучшил pyramid_match, но у shortest_path без модификаций результат выше)

Однако логарифмическое ядро для пары (pyramid_match, graphlet_sampling) дает лучший результат (0,54), чем они по отдельности 0,23 и 0,39

Скорость стала медленнее, тк помимо всего прочего нам надо считать по 2 логарифма для каждой комбинации

In [90]:
# делаем то же самое, что и для логарифмического, только теперь экспонента, структура класса такая же
class ExpKernel:
    def __init__(self, normalize=True, weights=None):
        self.normalize = normalize  # Нормализация ядер
        self.weights = weights if weights is not None else [0.5, 0.5]

    def fit_transform_pair(self, graphs, gk1, gk2):
        k1 = gk1.fit_transform(graphs)
        k2 = gk2.fit_transform(graphs)
        return np.exp(self.weights[0] * k1 + self.weights[1] * k2)

    def transform_pair(self, graphs, gk1, gk2):
        k1 = gk1.transform(graphs)
        k2 = gk2.transform(graphs)
        return np.exp(self.weights[0] * k1 + self.weights[1] * k2)

In [91]:
# выводим резы для экспоненциального ядра, такая же логика как для логарифмического
try:
    start = time()
    exp_kernel = ExpKernel(normalize=True)

    for i in range(len(kernels)):
        for j in range(i + 1, len(kernels)):
            gk1, gk2 = kernels[i], kernels[j]
            kernel1, kernel2 = kernel_names[i], kernel_names[j]

            K_train_custom = exp_kernel.fit_transform_pair(G_train, gk1, gk2)
            K_test_custom = exp_kernel.transform_pair(G_test, gk1, gk2)

            if not is_valid_data(K_train_custom) or not is_valid_data(K_test_custom):
                print(f"Пропущено ядро для ({kernel1}, {kernel2}) из-за некорректных значений.")
                continue

            clf = svm.SVC(kernel='precomputed', C=1, class_weight='balanced')
            clf.fit(K_train_custom, y_train)
            y_pred_custom = clf.predict(K_test_custom)
            acc_custom = accuracy_score(y_test, y_pred_custom)
            end = time()

            print(f"Exp Kernel Pair ({kernel1}, {kernel2}) -- Accuracy: {acc_custom}  time: {round(end - start, 2)} s")
except Exception as e:
    print(f"Ошибка: {e}")


Exp Kernel Pair (pyramid_match, graphlet_sampling) -- Accuracy: 0.5135135135135135  time: 10.76 s
Exp Kernel Pair (pyramid_match, weisfeiler_lehman) -- Accuracy: 0.4864864864864865  time: 11.8 s
Exp Kernel Pair (pyramid_match, shortest_path) -- Accuracy: 0.6216216216216216  time: 15.04 s
Exp Kernel Pair (graphlet_sampling, weisfeiler_lehman) -- Accuracy: 0.43243243243243246  time: 25.25 s
Exp Kernel Pair (graphlet_sampling, shortest_path) -- Accuracy: 0.6351351351351351  time: 38.25 s
Exp Kernel Pair (weisfeiler_lehman, shortest_path) -- Accuracy: 0.6216216216216216  time: 41.25 s


Скорость ниже по тем же причинам, что и у логарифмического
Опять же, улучшился результат только для пары (pyramid_match, graphtlet_sampling)
В остальном у нас weisfeiler_lehman все еще дает лучший результат без модификаций
Экспоненциальное ядро усиливает различия между значениями, поэтому у нас могут быть не очень хорошие результаты для ядер, которые чувствительны к большим перепадам в значениях. И, наоборот, экспоненциальное ядро дает лучший результат для ядер, которые лучше работают, когда есть различия между значениями
Все пары с shortest_path выдают результат > 0.62, поскольку shortest_path выдавал нам хороший результат и без модификаций, сейчас скор стал чуть хуже, но shortest_path улучшает accuracy для всех остальных ядер

Хочется проверить предположение о том, что ядра, которые выдают лучший результат при наличии больших различий между данными будут выдавать лучший скор с экспоненциальным ядром
Для этого посмотрю на экспоненциальное ядро для pyramid_match

In [94]:
start = time()
exp_kernel = ExpKernel(normalize=True)

gk1 = GraphKernel(kernel=[{"name": "pyramid_match"}], normalize=True)

K_train_custom = exp_kernel.fit_transform_pair(G_train, gk1, gk1)
K_test_custom = exp_kernel.transform_pair(G_test, gk1, gk1)

if not is_valid_data(K_train_custom) or not is_valid_data(K_test_custom):
    print("Пропущено из-за некорректных значений.")
else:
    clf = svm.SVC(kernel='precomputed', C=1, class_weight='balanced')
    clf.fit(K_train_custom, y_train)
    y_pred_custom = clf.predict(K_test_custom)
    acc_custom = accuracy_score(y_test, y_pred_custom)
    end = time()

    print(f"Accuracy: {acc_custom}  time: {round(end - start, 2)} s")

Accuracy: 0.36486486486486486  time: 1.52 s


Как видно результат и правда улучшился по сравнению с просто pyramid_match
Это произошло, тк pyramid_match работает хорошо, если у графов есть сходства в структуре или подграфах или если различия между графами одного класса малы, а различия между классами велики

При использовании экспоненциального ядра сходства усиливаются, а несходства наоборот, тк экспоненциальное ядро усиливает различия, то есть разделение на классы становится четче
Предположение оказалось правдой
